# Анализ базы данных сервиса для чтения книг

Нам предоставлен доступ к базе данных сервиса для чтения книг.

База данных состоит из нескольких таблиц, содержащих информацию о книгах, их авторах, издательствах, опубликовавших эти книги, а также информацию об оценках книг пользователями и отзывах о книгах.

Наша задача - проанализировать содержащуюся информацию, чтобы можно было сформировать предложение по подписке к этому сервису.

## Подключаем базу данных

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine

Устанавливаем доступ к базе данных

In [4]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


## Первичный анализ информации

Изучим какая информация содержится в базе данных, для этого посмотрим на первые строки в каждой таблице. 

### Таблица books

In [9]:
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


В данной таблице хранится информация о книгах, указаны автор произведения (`author_id`), название, количество страниц, дата публикации и издательство, выпустившее эту книгу (`publisher_id`).

In [12]:
query = '''SELECT COUNT( book_id) as books_count
FROM books
'''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,books_count
0,1000


In [14]:
query = '''SELECT MIN(publication_date), MAX(publication_date) 
FROM books
'''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,min,max
0,1952-12-01,2020-03-31


У нас хранится информация о 1000 книг, самая ранняя была выпущена в декабре 1952 года, а самая поздняя в марте 2020 года.

### Таблица publishers 

In [6]:
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
query = '''SELECT COUNT(publisher_id) 
FROM publishers
'''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,count
0,340


В этой таблице хранятся id  340-ка издательств и их названия.

### Таблица authors

In [8]:
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [9]:
query = '''SELECT COUNT(author_id) 
FROM authors
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,count
0,636


В этой таблице хранятся id 636-ти авторов книг.

### Таблица reviews

In [10]:
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


В данной таблице хранятся отзывы пользователей о различных книгах.

In [11]:
query = '''SELECT COUNT(DISTINCT username) FROM reviews'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,160


Отзывы оставили 160 пользователей.

### Таблица ratings

In [12]:
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Здесь хранятся все оценки пользователей, которые они ставили книгам.

In [13]:
query = '''SELECT COUNT(DISTINCT username) FROM ratings'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,160


Оценки тоже поставили только 160 пользователей.

## Задачи

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [14]:
query = '''SELECT COUNT(DISTINCT book_id) as book_count
FROM books
WHERE publication_date > '2000-01-01'
'''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,book_count
0,819


Мы знаем, что в базе хранятся 1000 книг, изданных с декабря 1952 года по март 2020. При этом, 819 книг из базы выпущены после 1 января 2000 года, что означает, что 181 книга из базы была издана не позднее 1 января 2000 года, что означает, что в основном в сервисе содержатся современные книги.

### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [15]:
query = '''
SELECT  b.book_id, 
        COUNT( DISTINCT r.review_id) AS reviews_count,
        AVG( DISTINCT rt.rating) AS rating_avg
FROM books AS b
LEFT JOIN reviews AS r ON b.book_id = r.book_id
LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
GROUP BY b.book_id
order by reviews_count
'''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,book_id,reviews_count,rating_avg
0,83,0,3.5
1,672,0,5.0
2,221,0,4.0
3,808,0,3.0
4,191,0,4.0
...,...,...,...
995,733,6,3.0
996,854,6,3.5
997,695,6,3.5
998,696,6,3.5


Сделаем проверку: посчитаем сколько всего отзывов хранится в базе данных

In [16]:
query = '''
SELECT  COUNT(r.review_id) 
FROM  reviews AS r 
'''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,count
0,2793


Сравним полученное значение с суммой по столбцу `reviews_count`.

In [17]:
query = '''with data as
(SELECT  b.book_id, 
        COUNT( DISTINCT r.review_id) AS reviews_count,
        AVG( DISTINCT rt.rating) AS rating_avg
FROM books AS b
LEFT JOIN reviews AS r ON b.book_id = r.book_id
LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
GROUP BY b.book_id
order by reviews_count)
select sum(reviews_count) from data
'''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,sum
0,2793.0


Видим, что суммы совпадают, значит, количество отзывов для каждой книги посчитано верно.

Каждая книга уникальна, пользователи оценивают их, ставя свои оценки, и пишут отзывы. На основании средних оценок можно составить рейтинг книг, а количество отзывов может показать наиболее неоднозначные, волнующие, затрагивающие, вызывающие определенные эмоции книги. Чем больше отзывов у книги, тем больше к ней интерес.

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [18]:
query = '''SELECT p.publisher,
COUNT(b.book_id) AS book_count
FROM books AS b
JOIN publishers AS p ON p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY  p.publisher
ORDER BY book_count DESC
LIMIT 1
'''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,book_count
0,Penguin Books,42


Издательства работают со многими авторами, и чем больше издано книг, тем крупнее издательство, тем с большим количеством авторов они могут работать.

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [19]:
query = '''WITH data AS
(
SELECT b.book_id,b.author_id
FROM books AS b
LEFT JOIN ratings AS rt ON b.book_id = rt.book_id 
GROUP BY b.book_id
HAVING COUNT(rt.rating) >= 50
),
data_1 AS
(
SELECT author_id, AVG(rt.rating) as rating_avg
FROM data
LEFT JOIN ratings AS rt ON data.book_id = rt.book_id 
GROUP BY author_id
ORDER BY rating_avg DESC
LIMIT 1
)
SELECT author
FROM authors
WHERE author_id = (SELECT author_id FROM data_1)
'''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author
0,J.K. Rowling/Mary GrandPré


Лучше всего оценили книги Джоан Роулинг с иллюстрациями Мэри Гранпре.

In [20]:
query = ''' WITH data AS
(SELECT b.book_id, avg(rt.rating) AS rating_avg
FROM books AS b
LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
GROUP BY b.book_id
HAVING COUNT(rt.rating)>=50
ORDER BY rating_avg DESC
LIMIT 1
)
SELECT a.author, ROUND(data.rating_avg,2)
FROM data
JOIN books AS b ON data.book_id = b.book_id
JOIN authors AS a ON b.author_id = a.author_id
'''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,round
0,J.K. Rowling/Mary GrandPré,4.41


Лучше всего оценили книги Джоан Роулинг с иллюстрациями Мэри Гранпре, средняя оценка их книг 4,41.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [21]:
query = '''WITH data AS
(
SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(rating_id)>48
),
data_1 AS
(
SELECT username, 
       COUNT(review_id) AS reviews_count
FROM reviews
WHERE username IN (SELECT * FROM data)
GROUP BY username
)
SELECT AVG(reviews_count)
FROM data_1
'''


con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


Активные пользователи, которые прочитали и оценили как минимум 48 книг, в среднем пишут 24 обзора на книги, то есть примерно к каждой второй прочитанной книге. Оценкам и обзорам таких пользователей вполне можно доверять, их обзоры можно выводить в числе первых.

## Вывод

Нам был предоставлен доступ к базе данных сервиса для чтения книг.

База состоит из пяти таблиц, которые содержат информацию о книгах, их авторах, издательствах, опубликовавших эти книги, а также информацию об оценках книг пользователями и отзывах о книгах.

Перед нами была поставлена задача проанализировать содержащуюся информацию, чтобы можно было сформировать предложение по подписке к этому сервису.

Для  начала мы изучили какую именно информацию содержат таблицы.

**Таблица `books`:**

В данной таблице хранится информация о 1000 книг изданных с декабря 1952 года по март 2020 года, указаны автор произведения (`author_id`), название, количество страниц, дата публикации и издательство, выпустившее эту книгу (`publisher_id`). 

**Таблица `publishers`:**

Хранит id и названия 340 издательств.

**Таблица `authors`:**

Хранит данные 636 авторов.

**Таблица `reviews`:**

Содержит 2793 отзыва пользователей на книги.

**Таблица `ratings`:**

содержит все оценки пользователей, которые они поставили книгам.

В результате анализа базы данных было замечено следующее:

- Не смотря на то, что сервис предоставляет книги, вышедшие с 1952 года, большинство же (819 книг) выпущены относительно недавно, не раньше 1 января 2000 года, что означает, что сервис нацелен на современных писателей. 

- Активных пользователей, которые пишут отзывы и ставят оценки прочитанным книгам, не так много, около 160. Это означает, что либо сервис не очень популярен и унего мало пользователей, либо он появился недавно и еще не набрал большое количество пользователей.

- У книг мало отзывов,  не больше семи (семь только у одной книги), что означает, что пользователи пока не видят необходимости делиться мнением о книгах. Можно предложить пользователям какой-нибудь бонус за отзывы к книгам в сервисе.

- В базе хранятся книги, изданные 340 издательствами, наибольшее число книг одного издательства - 42, что означает, что у большинства издательств в сервисе только по одной книге. Как известно, крупные издательства выпускают большее количество книг и работают с большим числом авторов, а следовательно у них большой отсев книг, которые могут не понравиться читателям.

- Можно посмотреть оценки книг, выпущенных малыми издательствами, и если они низкие, то стоит заменить такие книги на книги крупных издательств. Как правило, люди сначала ищут определенные популярные книги, а потом уже остаются в том или ином приложении, так как у них там сформировалась своя определенная библиотека, книги, которые они будут еще перечитывать. И, конечно, лучше предоставить им выбор из лучших книг.

- В базе книги 636 авторов, что с одной стороны хорошо, так как большой выбор авторов, а с другой, у каждого автора в сервисе в среднем 1-2 книги. Например, книги Джоан Роулинг оцениваются лучше всех, и поклонники этого автора хотели бы читать все ее книги в одном месте. Стоит посмотреть, книги каких авторов оцениваются лучше и добавить их книги в сервис.

- Активные пользователи, которые прочитали и оценили как минимум 48 книг, в среднем пишут 24 обзора на книги, то есть примерно к каждой второй прочитанной книге. Оценкам и обзорам таких пользователей вполне можно доверять, их обзоры можно выводить в числе первых.

Из всего вышеизложенного, можно сделать вывод, что поднимать вопрос о подписке сервиса пока рано, нужно направить усилия не только на привлечение читателя, но на то, чтобы он стал активным (оценивал книги, писал отзывы). Стоит пересмотреть базу книг, самые популярные книги можно продавать, что будет плюсом и для авторов, так как они будут также получать прибыль со своих книг.